In [34]:
%matplotlib inline
import pandas as pd
import datetime
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from tsfresh import extract_features
from tsfresh import extract_relevant_features
import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split
from sklearn.preprocessing import normalize

### Feature Extraction

In [2]:
data = pd.read_csv('../data/data_only_tsfresh_compatible.csv', names = ['x_acc', 'y_acc', 'z_acc', 'id'])
labels = pd.read_csv('../data/labels_only.csv', names = ['Blocking', 'Dodging', 'Inactive', 'Moving', 'Sprinting'])

In [3]:
extracted_features = extract_features(data, column_id = "id", column_sort = None, column_kind = None, column_value = None)

Feature Extraction: 100%|██████████| 10/10 [05:36<00:00, 28.76s/it]


In [4]:
print(extracted_features.shape)

(1068, 2382)


In [5]:
label_arr = labels.values
label_arr = np.argmax(label_arr, axis = 1)

We need to convert these labels also such that there is one label per example (right now there are 150 per example due to the way data was annotated). Also we need to convert the `extracted_features` dataframe into a NumPy array. This is required before attempting any sort of classification, of course.

#### Reducing the labels to the correct format

In [24]:
y_features = np.zeros(extracted_features.shape[0])
for i in range(len(label_arr)) : 
    if i % 150 == 0 : 
        y_features[i // 150] = label_arr[i]
        
# Also converting into Pandas Series for use in extracting relevant features using tsfresh
y = pd.Series(y_features, dtype = int)

In [ ]:
from tsfresh import select_features
from tsfresh.utilities.dataframe_functions import impute

impute(extracted_features)
features_filtered = select_features(extracted_features, y)

In [18]:
print(features_filtered.shape)

(1068, 693)


In [21]:
x_features = np.asarray(extracted_features)
print(x_features.shape)
x_features_relevant = np.asarray(features_filtered)
print(x_features_relevant.shape)

(1068, 2382)
(1068, 693)


### Separating the examples into training, validation and testing sets

In [48]:
x_train, x_test, y_train, y_test = train_test_split(x_features_relevant, y_features)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(801, 693)
(267, 693)
(801,)
(267,)


#### Normalizing the features

In [49]:
x_train_norm = normalize(x_train)
x_test_norm = normalize(x_test)

In [55]:
class_wt = {0 : 2, 1 : 2, 2 : 2, 3 : 1, 4 : 2}
svm_lin = RandomForestClassifier(n_estimators = 100, class_weight = 'balanced')
svm_lin.fit(x_train_norm, y_train)
y_pred = svm_lin.predict(x_test_norm)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
y_pred = svm_lin.predict(x_train_norm)
print(confusion_matrix(y_train, y_pred))
print(classification_report(y_train, y_pred))

[[ 0  0  6 24  2]
 [ 0  4  1 24  8]
 [ 0  1 21 32  4]
 [ 0  1  8 82 10]
 [ 0  1  0 22 16]]
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00        32
         1.0       0.57      0.11      0.18        37
         2.0       0.58      0.36      0.45        58
         3.0       0.45      0.81      0.58       101
         4.0       0.40      0.41      0.41        39

    accuracy                           0.46       267
   macro avg       0.40      0.34      0.32       267
weighted avg       0.43      0.46      0.40       267

[[ 97   0   0   0   0]
 [  0  82   0   0   0]
 [  0   0 152   0   3]
 [  0   1   3 305   1]
 [  0   1   0   0 156]]
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        97
         1.0       0.98      1.00      0.99        82
         2.0       0.98      0.98      0.98       155
         3.0       1.00      0.98      0.99       310
         4.0       0.97      0.99

/home/akshay/anaconda3/envs/pyt/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/akshay/anaconda3/envs/pyt/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/akshay/anaconda3/envs/pyt/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


There is some improvement in accuracy using both all features and only relevant features. But there is still some bias due to the unbalanced classes. So, need to re-try this using the balanced training set. 

#### TODO - (HIGH-PRIORITY)